# Klavis Notion MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Notion**:
- **Acquire** a Notion sandbox instance
- **Get** sandbox details  
- **Initialize** with databases and pages
- **Export** sandbox data
- **Reset** to initial state
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "notion"

## 2. Acquire Notion Sandbox

`POST /sandbox/notion` - Acquire an idle Notion sandbox instance (where we already setup Notion workspace and auth for you)

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}",
        headers=headers
    )

SANDBOX_ID = resp.json()["sandbox_id"]
print(resp.json())

## 3. Get Sandbox Details

`GET /sandbox` - Get sandbox info that you occupied

`GET /sandbox/notion/{sandbox_id}` - Get sandbox information

In [ ]:
# Get your sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox",
        headers=headers
    )

print(resp.json())

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

## 4. Initialize Notion Sandbox with Data

`POST /sandbox/notion/{sandbox_id}/initialize` - Initialize with databases and pages

In [ ]:
# Example Notion data (databases with pages)
notion_data = {
    "databases": [
        {
            "name": "Project Tracker",
            "summary": "Track team projects and their status",
            "display_icon": "🚀",
            "sources": [
                {
                    "label": "Projects",
                    "summary": "Project tracking table",
                    "fields": {
                        "Name": {"type": "title"},
                        "Status": {
                            "type": "select",
                            "options": ["Not Started", "In Progress", "Done"]
                        },
                        "Priority": {
                            "type": "select",
                            "options": ["High", "Medium", "Low"]
                        }
                    },
                    "pages": [
                        {
                            "name": "Website Redesign",
                            "attributes": {
                                "Status": "In Progress",
                                "Priority": "High"
                            },
                            "content_blocks": [
                                {"block_type": "paragraph", "content": "Redesign the company website with a modern look and feel."},
                                {"block_type": "heading_2", "content": "Goals"},
                                {"block_type": "bulleted_list_item", "content": "Improve page load speed by 50%"},
                                {"block_type": "bulleted_list_item", "content": "Mobile-first responsive design"},
                                {"block_type": "bulleted_list_item", "content": "Accessibility compliance (WCAG 2.1)"}
                            ],
                            "comments": [
                                {"content": "Design mockups are ready for review."}
                            ]
                        },
                        {
                            "name": "API Integration",
                            "attributes": {
                                "Status": "Not Started",
                                "Priority": "Medium"
                            },
                            "content_blocks": [
                                {"block_type": "paragraph", "content": "Integrate third-party APIs for payment and analytics."},
                                {"block_type": "to_do", "content": "Evaluate payment providers"},
                                {"block_type": "to_do", "content": "Set up analytics SDK"}
                            ],
                            "comments": []
                        }
                    ]
                }
            ]
        },
        {
            "name": "Meeting Notes",
            "summary": "Team meeting notes and action items",
            "display_icon": "📝",
            "sources": [
                {
                    "label": "Notes",
                    "summary": "Meeting notes archive",
                    "fields": {
                        "Title": {"type": "title"},
                        "Date": {"type": "date"},
                        "Status": {
                            "type": "select",
                            "options": ["Draft", "Final"]
                        }
                    },
                    "pages": [
                        {
                            "name": "Sprint Planning - Week 1",
                            "attributes": {
                                "Date": "2025-01-15",
                                "Status": "Final"
                            },
                            "content_blocks": [
                                {"block_type": "heading_2", "content": "Attendees"},
                                {"block_type": "paragraph", "content": "Alice, Bob, Charlie"},
                                {"block_type": "heading_2", "content": "Action Items"},
                                {"block_type": "numbered_list_item", "content": "Finalize design specs by Friday"},
                                {"block_type": "numbered_list_item", "content": "Set up CI/CD pipeline"},
                                {"block_type": "numbered_list_item", "content": "Schedule stakeholder demo"}
                            ],
                            "comments": [
                                {"content": "Great meeting, let's keep the momentum going!"}
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}

# Initialize the sandbox with the example Notion data
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/initialize",
        headers=headers,
        json=notion_data
    )

print(resp.json())

## 5. Export Notion Sandbox

`GET /sandbox/notion/{sandbox_id}/dump` - Export all databases and pages from the sandbox so you can see the current environment status!

In [ ]:
import json

async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/dump",
        headers=headers
    )

print(json.dumps(resp.json(), indent=2))

## 6. Reset Notion Sandbox

`POST /sandbox/notion/{sandbox_id}/reset` - Clear all databases and pages, for re-use this sandbox

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/reset",
        headers=headers
    )

print(resp.json())

## 7. Release Notion Sandbox

`DELETE /sandbox/notion/{sandbox_id}` - Release back to idle pool

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())